In [1]:
# Basic import
import os
import sys
import json
import math
import shutil
import random
import pandas as pd
import numpy as np

In [2]:
# Helper function
def writeProgress(msg, count, total):
    sys.stdout.write(msg + "{:.2%}\r".format(count/total))
    sys.stdout.flush()
    
def newPath(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def read_json(src_path):
    with open(src_path, 'r') as json_file:
        data = json.load(json_file)
    return data

def write_json(data,dst_path):
    with open(dst_path, 'w') as outfile:
        json.dump(data, outfile)

def writeLog(row):
    with open('log.txt', 'a') as outfile:
        outfile.write(row + '\n')

def getErrMsg(e):
    error_class = e.__class__.__name__ #取得錯誤類型
    detail = e.args[0] #取得詳細內容
    errMsg = "[{}] {}".format(error_class, detail)
    return errMsg

In [3]:
DATA_DIR = '../Baseline/'
pro_dir = os.path.join(DATA_DIR, '_pro_sg_colduser')
dmf_dir = os.path.join(DATA_DIR, '_DMF_data_colduser')
if not os.path.exists(pro_dir):
    os.makedirs(pro_dir)
if not os.path.exists(dmf_dir):
    os.makedirs(dmf_dir)

# Load numpy array

In [4]:
all_npy = np.load('./npy/all_2372.npy')
movie_genre = np.load('./npy/movie_genre.npy')
usr_following = np.load('./npy/user_followings.npy')
usr_genre = np.load('./npy/user_genre.npy')

print('All features:', all_npy.shape)
print('Movie genre:', movie_genre.shape)
print('User following:', usr_following.shape)
print('User genre:', usr_genre.shape)

All features: (165, 2372)
Movie genre: (165, 20)
User following: (1582, 165)
User genre: (1582, 20)


In [5]:
usr_nb = len(usr_following) # the number of users
movie_nb = len(movie_genre)  # the number of movies
print(usr_nb, movie_nb)

# usr_test_amount = 150
# movie_test_amount = 32
# print(usr_test_amount, movie_test_amount)

latent_dim = 64 # latent dims
ft_dim = all_npy.shape[1] # feature dims
embedding_dims = 240
print(latent_dim, ft_dim, embedding_dims)

1582 165
64 2372 240


# Training & testing split

## Prepare

In [6]:
#The number of following movie for each user
each_user = np.sum(usr_following, axis=1)
# print(each_user)

print('Min number of followings:', np.min(each_user))
print('Max number of followings:', np.max(each_user))
print('Avg of followers:', np.mean(each_user))

Min number of followings: 10
Max number of followings: 133
Avg of followers: 14.820480404551201


In [7]:
print('<= 15:', np.sum(each_user <= 15))
less_idx = np.nonzero(each_user <= 15)[0]
print(less_idx.shape, less_idx)

<= 15: 1142
(1142,) [   0    1    2 ... 1578 1580 1581]


In [8]:
for i in range(usr_nb):
    if not i in less_idx:
        each_user[i] = 0

In [9]:
print('Min number of followings:', np.min(each_user))
print('Max number of followings:', np.max(each_user))
print('Avg of followers:', np.mean(each_user))

Min number of followings: 0
Max number of followings: 15
Avg of followers: 8.36283185840708


In [10]:
(each_user.argsort()[::-1][:100])

array([ 326, 1448,  927,  926,  139,  895,  529,  117, 1403,  848,  545,
         99,  825,  824, 1436,  804,  953,  800,  570, 1472, 1473, 1478,
        606, 1486,   53, 1495, 1498, 1503,   43,   39, 1345, 1326, 1514,
       1043, 1155,  307, 1145,  314,  268, 1125, 1105,  357, 1223,  231,
       1235,  400, 1240, 1034,  464,  211,  408,  195,  428,  431, 1291,
        439,  981, 1303, 1311,  967,  460,  959,  730, 1156,  688,  676,
         13,  645,  679,  654,  633, 1571,  524,  675,   92,  822, 1225,
        873, 1290,  235, 1327,  561,  815, 1537,  188,  312,   81,  156,
         74, 1333,  423, 1451,  708, 1174, 1005,   93, 1382,  183, 1181,
       1553])

In [11]:
usr_idx = list(each_user.argsort()[::-1][:100])
print(usr_idx)

[326, 1448, 927, 926, 139, 895, 529, 117, 1403, 848, 545, 99, 825, 824, 1436, 804, 953, 800, 570, 1472, 1473, 1478, 606, 1486, 53, 1495, 1498, 1503, 43, 39, 1345, 1326, 1514, 1043, 1155, 307, 1145, 314, 268, 1125, 1105, 357, 1223, 231, 1235, 400, 1240, 1034, 464, 211, 408, 195, 428, 431, 1291, 439, 981, 1303, 1311, 967, 460, 959, 730, 1156, 688, 676, 13, 645, 679, 654, 633, 1571, 524, 675, 92, 822, 1225, 873, 1290, 235, 1327, 561, 815, 1537, 188, 312, 81, 156, 74, 1333, 423, 1451, 708, 1174, 1005, 93, 1382, 183, 1181, 1553]


In [12]:
train_idx = sorted(usr_idx[:80])
vad_idx = sorted(usr_idx[80:])

print(len(train_idx), train_idx)
print(len(vad_idx), vad_idx)

80 [13, 39, 43, 53, 92, 99, 117, 139, 195, 211, 231, 235, 268, 307, 314, 326, 357, 400, 408, 428, 431, 439, 460, 464, 524, 529, 545, 570, 606, 633, 645, 654, 675, 676, 679, 688, 730, 800, 804, 822, 824, 825, 848, 873, 895, 926, 927, 953, 959, 967, 981, 1034, 1043, 1105, 1125, 1145, 1155, 1156, 1223, 1225, 1235, 1240, 1290, 1291, 1303, 1311, 1326, 1345, 1403, 1436, 1448, 1472, 1473, 1478, 1486, 1495, 1498, 1503, 1514, 1571]
20 [74, 81, 93, 156, 183, 188, 312, 423, 561, 708, 815, 1005, 1174, 1181, 1327, 1333, 1382, 1451, 1537, 1553]


In [13]:
test_idx = sorted(usr_idx)
print(len(test_idx), test_idx)

100 [13, 39, 43, 53, 74, 81, 92, 93, 99, 117, 139, 156, 183, 188, 195, 211, 231, 235, 268, 307, 312, 314, 326, 357, 400, 408, 423, 428, 431, 439, 460, 464, 524, 529, 545, 561, 570, 606, 633, 645, 654, 675, 676, 679, 688, 708, 730, 800, 804, 815, 822, 824, 825, 848, 873, 895, 926, 927, 953, 959, 967, 981, 1005, 1034, 1043, 1105, 1125, 1145, 1155, 1156, 1174, 1181, 1223, 1225, 1235, 1240, 1290, 1291, 1303, 1311, 1326, 1327, 1333, 1345, 1382, 1403, 1436, 1448, 1451, 1472, 1473, 1478, 1486, 1495, 1498, 1503, 1514, 1537, 1553, 1571]


In [14]:
usr_nb = len(test_idx)
usr_test_amount = len(test_idx)
movie_test_amount = 32 #math.floor(len(less_idx)*0.5)
print(usr_nb, usr_test_amount, movie_nb, movie_test_amount)

100 100 165 32


# Build UserFollowingRecord.csv

In [15]:
user = []
movie = []

for i in test_idx:
    for j in range(movie_nb):
        if usr_following[i][j] == 1:
            # build df
            user.append(test_idx.index(i))
            movie.append(j)

In [16]:
df = pd.DataFrame(data={'userId': user, 'movieId': movie})
df

,userId,movieId
0,0,1
1,0,12
2,0,13
3,0,21
4,0,22
...,...,...
1467,99,104
1468,99,121
1469,99,128
1470,99,154


In [17]:
df.to_csv(os.path.join(pro_dir, 'UserFollowingRecord.csv'), index = 0)

In [18]:
unique_uid = pd.unique(df['userId'])
unique_uid.shape

(100,)

In [19]:
unique_sid = pd.unique(df['movieId'])
unique_sid.shape

(159,)

## Setup 

In [20]:
# init
random.seed(42)
test_te_uid = []
test_te_sid= []

train_uid = []
train_sid = []
vad_uid = []
vad_sid = []

for i in test_idx:
    # init
    t_for_train = []
    t_for_test = []
    
    temp_t = []
    for j in range(movie_nb):
        if usr_following[i][j] == 1:
            temp_t.append(j)
            
    t_for_test = random.sample(temp_t, math.ceil(0.5*len(temp_t)))
    for k in t_for_test:
        test_te_uid.append(i)
        test_te_sid.append(k)
    
    t_for_train = [item for item in temp_t if not item in t_for_test]
    for k in t_for_train:
        if i in train_idx:
            train_uid.append(i)
            train_sid.append(k)
        elif i in vad_idx:
            vad_uid.append(i)
            vad_sid.append(k)
        else:
            print('Error!!!')

In [21]:
print('The length of test_te:',len(test_te_uid), len(test_te_sid))
print('The length of train:',len(train_uid), len(train_sid))
print('The length of vad:',len(vad_uid), len(vad_sid))

The length of test_te: 772 772
The length of train: 560 560
The length of vad: 140 140


# Build 1 on 1 dataframe

In [22]:
train = pd.DataFrame(data={'uid': train_uid, 'sid': train_sid}, columns=['uid', 'sid'])
train

,uid,sid
0,13,37
1,13,50
2,13,64
3,13,66
4,13,141
...,...,...
555,1571,101
556,1571,104
557,1571,128
558,1571,154


In [23]:
vad = pd.DataFrame(data={'uid': vad_uid, 'sid': vad_sid}, columns=['uid', 'sid'])
vad

,uid,sid
0,74,53
1,74,64
2,74,105
3,74,124
4,74,125
...,...,...
135,1553,44
136,1553,68
137,1553,111
138,1553,114


In [24]:
test_plays_tr = train.sample(frac=0.2, random_state=42)
test_plays_tr

,uid,sid
453,1303,87
341,959,84
177,529,84
86,268,75
332,953,119
...,...,...
88,268,102
235,676,118
352,981,57
33,92,150


In [25]:
train = train.drop(test_plays_tr.index)
train

,uid,sid
0,13,37
1,13,50
3,13,66
4,13,141
5,13,147
...,...,...
552,1514,156
554,1571,9
557,1571,128
558,1571,154


In [26]:
test = pd.DataFrame(data={'uid': test_te_uid, 'sid': test_te_sid}, columns=['uid', 'sid'])
test_plays_te = test
test_plays_te

,uid,sid
0,13,93
1,13,12
2,13,1
3,13,99
4,13,22
...,...,...
767,1571,102
768,1571,44
769,1571,81
770,1571,11


In [27]:
unique_uid = np.concatenate((pd.unique(train['uid']), pd.unique(vad['uid'])), axis=0)
print(unique_uid.shape)
unique_uid

(100,)


array([  13,   39,   43,   53,   92,   99,  117,  139,  195,  211,  231,
        235,  268,  307,  314,  326,  357,  400,  408,  428,  431,  439,
        460,  464,  524,  529,  545,  570,  606,  633,  645,  654,  675,
        676,  679,  688,  730,  800,  804,  822,  824,  825,  848,  873,
        895,  926,  927,  953,  959,  967,  981, 1034, 1043, 1105, 1125,
       1145, 1155, 1156, 1223, 1225, 1235, 1240, 1290, 1291, 1303, 1311,
       1326, 1345, 1403, 1436, 1448, 1472, 1473, 1478, 1486, 1495, 1498,
       1503, 1514, 1571,   74,   81,   93,  156,  183,  188,  312,  423,
        561,  708,  815, 1005, 1174, 1181, 1327, 1333, 1382, 1451, 1537,
       1553])

In [28]:
# unique_sid = unique_sid
with open(os.path.join(pro_dir, 'unique_sid.txt'), 'w') as f:
    for sid in unique_sid:
        f.write('%s\n' % sid)
print(unique_sid.shape)
unique_sid

(159,)


array([  1,  12,  13,  21,  22,  37,  50,  64,  66,  87,  93,  99, 141,
       147, 158,   0,  11,  30,  40,  42,  49,  57, 101, 102, 106, 123,
       126, 128, 138, 144,  19,  23,  45,  59,  69,  72,  81,  88, 111,
       120, 133, 137, 146,   8,  25,  39,  70,  80,  95,  98, 156,  53,
        55,  76, 103, 105, 124, 125, 154, 161,  28,  97, 112, 140,  34,
        44,  58, 129, 150,  79,  90, 132, 152, 163,   4,   9,  31,  60,
        62, 148,  10,  77,  96, 131, 149,  68, 122, 134, 164,  20,  24,
       116, 119,  41,  52,  78,  92, 127,   2,  14,  91, 135,  29,  67,
        86, 145,  84,  75, 159,  63, 136,  89,   3,  26, 139,  32,  33,
        35,  71, 153, 108, 118, 142,  47, 117,  36, 114,  46,  61,  82,
       104, 107,  83, 100,  18, 143,  74,   5,  17, 109, 155,  48,  85,
       157,  15,  43, 151, 121,  51,  94,  16,   6,  38,  73, 110, 130,
       115,  54,  56])

In [29]:
show2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))
profile2id = dict((pid, i) for (i, pid) in enumerate(unique_uid))

In [30]:
print(len(unique_sid), len(unique_uid))
print(len(show2id), len(profile2id))

159 100
159 100


In [31]:
def split_train_test_proportion(data, test_prop=0.2, lower_bound=5):
    data_grouped_by_user = data.groupby('uid')
    tr_list, te_list = list(), list()

    np.random.seed(98765)

    for i, (_, group) in enumerate(data_grouped_by_user):
        n_items_u = len(group)
        
        if n_items_u >= lower_bound:
            idx = np.zeros(n_items_u, dtype='bool')
            idx[np.random.choice(n_items_u, size=int(test_prop * n_items_u), replace=False).astype('int64')] = True

            tr_list.append(group[np.logical_not(idx)])
            te_list.append(group[idx])
        else:
            tr_list.append(group)

        if i % 100 == 0:
            print("%d users sampled" % i)
            sys.stdout.flush()
    
    data_tr = pd.concat(tr_list)
    data_te = pd.concat(te_list)

    return data_tr, data_te

In [32]:
vad_plays_tr, vad_plays_te = split_train_test_proportion(vad, test_prop=0.5, lower_bound=2)
print(vad_plays_tr.shape, vad_plays_te.shape)

0 users sampled
(80, 2) (60, 2)


In [33]:
# test_plays_tr, test_plays_te = split_train_test_proportion(test)
# print(test_plays_tr.shape, test_plays_te.shape)

In [34]:
def numerize(tp):
    uid = list(map(lambda x: profile2id[x], tp['uid']))
    sid = list(map(lambda x: show2id[x], tp['sid']))
    return pd.DataFrame(data={'uid': uid, 'sid': sid}, columns=['uid', 'sid'])

In [35]:
train_data = numerize(train)
train_data.to_csv(os.path.join(pro_dir, 'train.csv'), index=False)
vad_data_tr = numerize(vad_plays_tr)
vad_data_tr.to_csv(os.path.join(pro_dir, 'validation_tr.csv'), index=False)
vad_data_te = numerize(vad_plays_te)
vad_data_te.to_csv(os.path.join(pro_dir, 'validation_te.csv'), index=False)
test_data_tr = numerize(test_plays_tr)
test_data_tr.to_csv(os.path.join(pro_dir, 'test_tr.csv'), index=False)
test_data_te = numerize(test_plays_te)
test_data_te.to_csv(os.path.join(pro_dir, 'test_te.csv'), index=False)

# For DMF

## Chilli

In [36]:
user = []
movie = []
score = []
less_idx = list(less_idx)

for i in test_idx:
    for j in range(movie_nb):
        if usr_following[i][j] == 1:
            # build df
            user.append(test_idx.index(i))
            movie.append(j)
            score.append(1)

In [37]:
df = pd.DataFrame(data={'user': user, 'items': movie, 'ratings': score})
#                   columns=['user', 'movie', 'score', 'time'])
df

,user,items,ratings
0,0,1,1
1,0,12,1
2,0,13,1
3,0,21,1
4,0,22,1
...,...,...,...
1467,99,104,1
1468,99,121,1
1469,99,128,1
1470,99,154,1


In [38]:
print(pd.unique(df['user']).shape, pd.unique(df['items']).shape)

(100,) (159,)


In [39]:
df.to_csv(os.path.join(dmf_dir, 'myratings_cuser.dat')) #, header=None, index=None, sep=':')